In [1]:
%matplotlib widget

import tensorflow as tf
import numpy as np
import ipympl

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.decomposition import PCA

In [2]:
LEARNING_RATE = 0.001
MOMENTUM = 0.8
TRAINING_STEPS = 2000
BATCH_SIZE = 128
NUM_SAMPLES = 3

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [4]:
x_train = x_train.astype(np.float32)
mean = x_train.mean()
std = x_train.std()

x_train -= mean
x_train /= std

x_train = x_train[..., np.newaxis]

In [5]:
plt.imshow(x_train[0, ..., 0].astype(np.float32), cmap='gray')

FigureCanvasNbAgg()

In [6]:
train_dict = {}
for x, y in zip(x_train, y_train):
    if y in train_dict:
        train_dict[y].append(x)
    else:
        train_dict[y] = [x]

In [7]:
def random_samples(source, num_samples):
    assert num_samples > 0
    source_size = len(source)
    if num_samples < source_size:
        indices = []
        while len(indices) < num_samples:
            index = np.random.randint(source_size)
            if index not in indices:
                indices.append(index)
                
        indices = np.array(indices)
    else:
        indices = np.random.randint(source_size, size=num_samples)
        
    indices = indices.astype(np.int32)
    return [source[i] for i in indices]

In [8]:
def py_map(i):
    anchors = train_dict[i]
    samples = random_samples(anchors, 2 * NUM_SAMPLES)
    anchor = samples[:NUM_SAMPLES]
    positive = samples[NUM_SAMPLES:]
    
    negative_class = i
    while negative_class == i:
        negative_class = np.random.randint(len(train_dict.keys()))
        
    negatives = train_dict[negative_class]
    negative = random_samples(negatives, NUM_SAMPLES)
    return anchor, positive, negative, np.int32(i), np.int32(negative_class)    

In [9]:
# def py_map(i):
#     anchors = train_dict[i]
#     anchor_index = np.random.randint(len(anchors))
#     anchor = anchors[anchor_index]
    
#     positive_index = anchor_index
#     while positive_index == anchor_index:
#         positive_index = np.random.randint(len(anchors))
#     positive = anchors[positive_index]
    
#     negative_class = i
#     while negative_class == i:
#         negative_class = np.random.randint(len(train_dict.keys()))

#     negatives = train_dict[negative_class]
#     negative = negatives[np.random.randint(len(negatives))]
    
#     return anchor, positive, negative, np.int32(i), np.int32(negative_class)

def _map(i):
    anchor, positive, negative, pos_class, neg_class = tf.py_func(py_map, [i], [tf.float32, tf.float32, tf.float32, tf.int32, tf.int32])
    anchor.set_shape((None, 28, 28, 1))
    positive.set_shape((None, 28, 28, 1))
    negative.set_shape((None, 28, 28, 1))
    
    return anchor, positive, negative, pos_class, neg_class

# def _map(i):
#     anchor, positive, negative, pos_class, neg_class = tf.py_func(py_map, [i], [tf.float32, tf.float32, tf.float32, tf.int32, tf.int32])
#     anchor.set_shape((28, 28, 1))
#     positive.set_shape((28, 28, 1))
#     negative.set_shape((28, 28, 1))
    
#     return anchor, positive, negative, pos_class, neg_class

def _batch_map(anchor, positive, negative, pos_class, neg_class):
    anchor = tf.reshape(anchor, (-1, 28, 28, 1))
    positive = tf.reshape(positive, (-1, 28, 28, 1))
    negative = tf.reshape(negative, (-1, 28, 28, 1))
    
    return anchor, positive, negative, pos_class, neg_class

In [10]:
def model(inputs):
    conv1 = tf.layers.Conv2D(32, 3, activation=tf.nn.relu, padding='same')(inputs)
    conv2 = tf.layers.Conv2D(64, 2, activation=tf.nn.relu, padding='same')(conv1)
    max_pool1 = tf.layers.MaxPooling2D(2, 2)(conv2)

    conv3 = tf.layers.Conv2D(128, 3, activation=tf.nn.relu, padding='same')(max_pool1)
    conv4 = tf.layers.Conv2D(256, 3, activation=tf.nn.relu, padding='same')(conv3)
    max_pool2 = tf.layers.MaxPooling2D(2, 2)(conv4)

    conv5 = tf.layers.Conv2D(256, 3, activation=tf.nn.relu, padding='same')(max_pool2)
    conv6 = tf.layers.Conv2D(256, 3, activation=tf.nn.relu, padding='same')(conv5)
    max_pool3 = tf.layers.MaxPooling2D(2, 2)(conv6)

    flatten = tf.layers.Flatten()(max_pool3)
    embeddings = tf.layers.Dense(32, activation=tf.sigmoid)(flatten)
    
    with tf.variable_scope('visual_control'):
        alpha = tf.layers.Dense(1, activation=tf.sigmoid)(embeddings)
        alpha = tf.reshape(alpha, (-1, NUM_SAMPLES, 1))
        v = tf.reshape(embeddings, (-1, NUM_SAMPLES, 32))
        vm = tf.reduce_sum(v * alpha, axis=1, keepdims=True) / tf.reduce_sum(alpha, axis=1, keepdims=True)
        vm = tf.tile(vm, [1, NUM_SAMPLES, 1])
        
    with tf.variable_scope('content_control'):
        conc_v = tf.concat([v, vm], axis=-1)
        conc_v = tf.reshape(conc_v, (-1, 2 * 32))
        betta = tf.layers.Dense(1, activation=tf.sigmoid)(conc_v)
        betta = tf.reshape(betta, (-1, NUM_SAMPLES, 1))
        
    weights = alpha * betta
    
    embeddings = tf.reduce_sum(v * weights, axis=1) / tf.reduce_sum(weights, axis=1)
    return embeddings
#     normalized_embeddings = embeddings / tf.norm(embeddings, ord=2, axis=-1, keepdims=True)
#     return normalized_embeddings

In [11]:
def triplet_loss(anchor, positive, negative, eps=1e-7):
    with tf.name_scope('triplet_loss'):
        pos_norm = tf.norm(anchor-positive, ord=2, axis=-1)
        neg_norm = tf.norm(anchor-negative, ord=2, axis=-1)
        loss = tf.maximum(tf.square(pos_norm) - tf.square(neg_norm) + eps, 0)
        return tf.reduce_mean(loss)
    
def nonlinear_triplet_loss(anchor, positive, negative, eps=1e-7, beta=32):
    with tf.name_scope('mse_triplet_loss'):
        pos = -tf.square(anchor - positive) / beta + 1 + eps
        neg = -(beta - tf.square(anchor - negative)) / beta + 1 + eps
        
        pos_log = tf.log(pos)
        neg_log = tf.log(neg)
        
        loss = tf.reduce_sum(pos_log - neg_log, axis=-1)
        return tf.reduce_mean(loss)  

In [12]:
dataset = tf.data.Dataset.range(len(train_dict.keys()))
dataset = dataset.apply(tf.contrib.data.shuffle_and_repeat(buffer_size=500))
dataset = dataset.map(_map, 6)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.map(_batch_map, 2)

train_iterator = dataset.make_one_shot_iterator()
batch = train_iterator.get_next()

anchor = batch[0]
positive = batch[1]
negative = batch[2]
pos_classes = batch[3]
neg_classes = batch[4]

In [13]:
with tf.name_scope('anchor'):
    with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):    
        anchor_embeddings = model(anchor)
        
with tf.name_scope('positive'):
    with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE): 
        positive_embeddings = model(positive)
        
with tf.name_scope('negative'):
    with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE): 
        negative_embeddings = model(negative)

In [14]:
with tf.name_scope('test'):
    with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):   
        features = tf.placeholder(tf.float32, [None, 28, 28, 1])
        embeddings = model(features)

In [15]:
loss = nonlinear_triplet_loss(anchor_embeddings, positive_embeddings, negative_embeddings)

In [16]:
with tf.name_scope('training'):
    optimizer = tf.train.MomentumOptimizer(LEARNING_RATE, MOMENTUM)
    train_op = optimizer.minimize(loss)

In [17]:
sess = tf.Session()
summary_writer = tf.summary.FileWriter('logdir', sess.graph)

sess.run(tf.global_variables_initializer())
    
losses = []
for i in tqdm(range(TRAINING_STEPS)):
    _loss, _ = sess.run([loss, train_op])
    losses.append(_loss)

100%|██████████| 2000/2000 [05:18<00:00,  6.27it/s]


In [18]:
plt.figure()
plt.plot(losses)

FigureCanvasNbAgg()

In [19]:
x_test = x_test.astype(np.float32)
x_test -= mean
x_test /= std

In [20]:
x_test = x_test[..., np.newaxis].astype(np.float16)

In [21]:
test_dict = {}
for x, y in zip(x_test, y_test):
    if y in test_dict:
        test_dict[y].append(x)
    else:
        test_dict[y] = [x]

In [22]:
test_data = []
test_labels = []
for key, val in test_dict.items():
    for i in range(0, len(val)-NUM_SAMPLES+1, NUM_SAMPLES):
        test_data.append(val[i:i+NUM_SAMPLES])
        test_labels.append(key)

In [23]:
test_data = np.array(test_data).reshape(-1, 28, 28, 1)

In [24]:
_embeddings = sess.run(embeddings, feed_dict={features: test_data})

In [25]:
_embeddings = _embeddings.astype(np.float32)

In [26]:
pca = PCA(n_components=3)
pca.fit(_embeddings)
new_embeddings = pca.transform(_embeddings) 
# new_embeddings = _embeddings

In [27]:
sample_indices = np.random.randint(0, len(test_labels), 500, dtype=np.int32)

In [28]:
test_labels = np.array(test_labels)[sample_indices]

In [29]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
for i in range(10):
    indices = np.argwhere(np.array(test_labels) == i)[:, 0]
    ax.scatter(new_embeddings[sample_indices][indices, 0], 
               new_embeddings[sample_indices][indices, 1], 
               new_embeddings[sample_indices][indices, 2], 
               label=i)
ax.legend()

FigureCanvasNbAgg()

In [41]:
_x_test = x_test[np.newaxis, ...]
_x_test = np.transpose(_x_test, (1, 0, 2, 3, 4))
_x_test = np.tile(_x_test, (1, NUM_SAMPLES, 1, 1, 1))
_x_test = np.reshape(_x_test, (-1, 28, 28, 1))

In [42]:
_embeddings = sess.run(embeddings, feed_dict={features: _x_test})
_embeddings = _embeddings.astype(np.float32)

In [43]:
pca = PCA(n_components=3)
pca.fit(_embeddings)
new_embeddings = pca.transform(_embeddings) 

In [46]:
sample_indices = np.random.randint(0, len(y_test), 1000, dtype=np.int32)
_y_test = y_test[sample_indices]
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
for i in range(10):
    indices = np.argwhere(np.array(_y_test) == i)[:, 0]
    ax.scatter(new_embeddings[sample_indices][indices, 0], 
               new_embeddings[sample_indices][indices, 1], 
               new_embeddings[sample_indices][indices, 2], 
               label=i)
ax.legend()

FigureCanvasNbAgg()